In [ ]:
import pandas as pd
from snowflake.connector.pandas_tools import write_pandas

In [ ]:
%run logging.ipynb

In [ ]:
class SnowflakeHandlerRaw(SnowflakeConnector):
    """
    Classe para gerenciar a conexão e operações com Snowflake.

    Atributos:
    ----------
    user: str
        Nome do usuário para a conexão com Snowflake.
    password: str
        Senha do usuário para a conexão com Snowflake.
    account: str
        Identificador da conta Snowflake.
    warehouse: str
        Nome do warehouse a ser usado na conexão.
    database: str
        Nome do banco de dados a ser usado na conexão.
    schema: str
        Nome do schema a ser usado na conexão.
    """

    def __init__(self):
        """
        Inicializa a classe SnowflakeHandler carregando as variáveis de ambiente.
        """
        super().__init__()
        self.logger_class = Logger()
        self.logger = self.logger_class.set_logger()        

    def save_dataframe(self, df: pd.DataFrame, table_name: str, column_names: list) -> None:
        """
        Salva um DataFrame no Snowflake, sobrescrevendo o conteúdo da tabela.

        Parâmetros:
        -----------
        df: pd.DataFrame
            O DataFrame que será salvo no Snowflake.
        table_name: str
            O nome da tabela onde o DataFrame será salvo.
        column_names: list
            Lista de nomes das colunas que devem ser usadas no DataFrame.
        
        Retorna:
        --------
        None
        """
        super().connect()

        if not self.connection or not self.cursor:
            raise RuntimeError("Conexão com Snowflake não estabelecida.")
             
        df.columns = column_names

        # Limpar aspas e espaços desnecessários dos nomes das colunas
        df.columns = df.columns.str.replace(r'^"|"$', '', regex=True).str.strip()

        # Apagar o conteúdo existente na tabela
        try:
            self.cursor.execute(f"DELETE FROM {table_name}")
            self.logger.info(f"Conteúdo da tabela '{table_name}' deletado com sucesso.")
        except Exception as e:
            self.logger.error(f"Erro ao deletar o conteúdo da tabela '{table_name}': {e}")
            raise

        # Inserir o novo DataFrame na tabela
        success, nchunks, nrows, _ = write_pandas(self.connection, df, table_name)

        if success:
            self.logger.info(f"DataFrame salvo com sucesso na tabela '{table_name}'. Total de linhas: {nrows}.")
        else:
            self.logger.error(f"Falha ao salvar o DataFrame na tabela '{table_name}'.")

        super().disconnect()
